# NDVI Trends

In [3]:
# Import required libraries
import odc.stac
import pandas as pd
import planetary_computer
import pystac_client
import xarray as xr
import hvplot.xarray
import panel as pn
from shapely.geometry import box
import geopandas as gpd
import rioxarray
import numpy as np
from rasterio.mask import mask

ImportError: cannot import name 'quote' from 'dask.base' (c:\Users\kathr\clean-and-green-philly\.venv\Lib\site-packages\dask\base.py)

## Motivation
Nissim has pointed out potentiality for time series, which I decided to investigate here.
I wrote the code under his guidance.

## Process

In [ ]:
def get_datasets(catalog, bbox:list[float], datetime:str, cloudy_less_than:float):
	# Load all selected items (tiles) into a list of datasets
	bands_of_interest = ["red", "green", "blue", "nir", "swir16"]
	datasets = []
	
	search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": cloudy_less_than}}
	)

	items = search.items()
	items = list(items)

	for item in items:

		ds_tile = odc.stac.stac_load(
			items=[item],
			bands=bands_of_interest,
			bbox=bbox,
			resolution=5,
			chunks={},  # Enable Dask for memory efficiency
		)
		datasets.append(ds_tile)

		# Load all selected items (tiles) into a list of datasets
	bands_of_interest = ["red", "green", "blue", "nir", "swir16"]
	datasets = []

	for item in items:
		ds_tile = odc.stac.stac_load(
			items=[item],
			bands=bands_of_interest,
			bbox=bbox,
			resolution=5,
			chunks={},  # Enable Dask for memory efficiency
		)
		datasets.append(ds_tile)
	print("=== completed dataset collection ===")
	return xr.concat(datasets, dim="time")

In [ ]:
def calculate_NDVI(nir, red):
    """
    Calculate the NDVI from the NIR and red landsat bands
    """

    if red.max() > 1:
        red = red / 10000.0
    if nir.max() > 1:
        nir = nir / 10000.0

    # Calculate NDVI
    ndvi = (nir - red) / (nir + red)

    # Mask invalid values (divide by zero or NaN)
    # ndvi = np.nan_to_num(ndvi, nan=-9999)  # Replace NaN with a placeholder
    
    # Return
    return np.nanmedian(ndvi)

# NDVI_city = calculate_NDVI(nir, red)
# median_NDVI_city = np.nanmedian(NDVI_city)
# median_NDVI_city

I investigated using the city's median NDVI index using time.

In [ ]:
city_limits = gpd.read_file("./City_Limits.geojson")
city_limits = city_limits.to_crs(32618)

In [ ]:
def ndvi_trends(datasets, clip=city_limits):
	timestamps = datasets.time.values


	res = []
	for timestamp in timestamps:
		print(f"=== Median NDVI calculated for {timestamp} ===")
		query = datasets.sel(time=timestamp).to_array(dim="band").compute()
		query = query.rio.clip([clip], query.rio.crs, drop=True)
		red = query.red.values
		nir = query.nir.values	
		
		res.append(calculate_NDVI(nir, red))
		
		print(f"=== collection complete for {timestamp} ===")
	
	return pd.DataFrame({'timestamp': timestamps, 'median ndvi': res})


In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,  # Automatically signs requests
)
# List available collections
all_collections = [i.id for i in catalog.get_collections()]
sentinel_collections = [collection for collection in all_collections if "sentinel" in collection]
print("Available Sentinel Collections:", sentinel_collections)

# Corrected Query for Sentinel-2 with Cloud Cover Filter
bbox = [-75.2803, 39.8670, -74.9557, 40.1379]  # Philadelphia bounding box
datetime = "2022-08-01/2024-07-31"  # Summer 2024
cloudy_less_than = 10  # Percent cloud cover threshold

Took a while, but I did found evidence to suggest that there are fluctuations here after computing the ndvi for each point for the collection

In [ ]:
datasets = get_datasets(catalog=catalog, bbox=bbox, datetime=datetime, cloudy_less_than=cloudy_less_than)
trends = ndvi_trends(datasets=datasets)
trends

In [ ]:
trends['timestamp'] = datasets.time.values

In [ ]:
trends.to_parquet('ndvi_trends.parquet')

In [ ]:
trends.drop(columns=['time'], inplace=True)
# trends.columns

In [ ]:
trends

Plot seems to indicate this to be the case.

In [ ]:
trends.plot.scatter('timestamp', 'ndvi')

So, I fit a quick sinodal curve and plotted it as show below.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# 1. Define your sinusoidal model
def seasonal_sin(t, offset, amplitude, phase):
    """
    Simple seasonal sine function:
      NDVI(t) = offset + amplitude * sin((2*pi/period)*t + phase)
    """
    period = 365.0  # days, if you expect a 1-year cycle
    return offset + amplitude * np.sin((2 * np.pi / period) * t + phase)

# 2. Convert your timestamps to a numeric scale
#    Suppose 'trends' is a pandas DataFrame with columns 'timestamp' (datetime) and 'ndvi'
trends['t_ordinal'] = trends['timestamp'].map(pd.Timestamp.toordinal)
x = trends['t_ordinal'].values.astype(float)
y = trends['ndvi'].values.astype(float)

# 3. Provide an initial guess for offset, amplitude, and phase
initial_guess = [0.2, 0.1, 0.0]  # for example

# 4. Fit the model
popt, pcov = curve_fit(seasonal_sin, x, y, p0=initial_guess)
offset, amplitude, phase = popt
print("Fitted parameters:")
print("offset  =", offset)
print("amplitude =", amplitude)
print("phase    =", phase)

# 5. Generate fitted values
trends['ndvi_sine_fit'] = seasonal_sin(x, offset, amplitude, phase)

# 6. Plot the results
plt.figure(figsize=(10, 5))
plt.scatter(trends['timestamp'], trends['ndvi'], label='Data', s=20)
plt.plot(trends['timestamp'], trends['ndvi_sine_fit'], color='red', label='Sine Fit')
plt.xlabel("Date")
plt.ylabel("NDVI")
plt.title("Seasonal NDVI Fit (Sine Wave)")
plt.legend()
plt.show()


## Conclusion
- Definitely, a more sinodal pattern here. Potentially, a periodic time series?
- Computation of median ndvi is a drag on the system but not to much at the moment.